In [1]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib
from nltk.tokenize import RegexpTokenizer
import nltk
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix 
from imblearn.combine import SMOTETomek
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/flaviaferrusmarimon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#pip install imblearn

In [3]:
stop_words = set(stopwords.words("english"))
default_stemmer = PorterStemmer()
default_stopwords = stopwords.words('english')
default_tokenizer=RegexpTokenizer(r"\w+")

In [4]:
data = pd.read_csv('./train.csv')
data.head() 

,Id,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,required_doughnuts_comsumption,fraudulent
0,0,5737,Jr. Business Analyst & Quality Analyst (entry...,"US, NJ, PISCATAWAY",NaN,NaN,NaN,"Duration: Full time / W2Location: Piscataway,N...",What we require:-- Masters degree in Computers...,NaN,0,0,0,Full-time,Entry level,Master's Degree,Financial Services,Finance,-0.410911,0
1,1,7107,English Teacher Abroad,"US, PA, Scranton",NaN,NaN,We help teachers get safe &amp; secure jobs ab...,"Play with kids, get paid for it Love travel? J...",University degree required. TEFL / TESOL / CEL...,See job description,0,1,1,Contract,NaN,Bachelor's Degree,Education Management,NaN,-0.163785,0
2,2,11979,SQL Server Database Developer Job opportunity ...,"US, IL, Barrington",NaN,90000-100000,We are an innovative personnel-sourcing firm w...,Position : SQL Server Database DeveloperJob Lo...,Position : SQL Server Database DeveloperJob Lo...,Benefits - FullBonus Eligible - Yes,0,0,0,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,-0.244369,0
3,3,9375,Legal Analyst - 12 Month FTC,"GB, LND, London",Legal,NaN,MarketInvoice is one of the most high-profile ...,DescriptionOur mission at MarketInvoice is to ...,Duties and ResponsibilitiesReviewing contractu...,Competitive salaryPrivate HealthcareHalf price...,0,1,0,Full-time,Associate,Professional,Financial Services,Legal,-0.456807,0
4,4,1301,Part-Time Finance Assistant,"GB, LND,",NaN,NaN,NaN,Salary:£9 - £10 per hour We are currently goin...,"Your role will be a varied, interesting and in...",Salary:£9 - £10 per hour,0,0,0,Part-time,NaN,NaN,Accounting,NaN,-0.217880,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10728 entries, 0 to 10727
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Id                              10728 non-null  int64  
 1   job_id                          10728 non-null  int64  
 2   title                           10728 non-null  object 
 3   location                        10523 non-null  object 
 4   department                      3719 non-null   object 
 5   salary_range                    1709 non-null   object 
 6   company_profile                 8766 non-null   object 
 7   description                     10727 non-null  object 
 8   requirements                    9099 non-null   object 
 9   benefits                        6360 non-null   object 
 10  telecommuting                   10728 non-null  int64  
 11  has_company_logo                10728 non-null  int64  
 12  has_questions                   

In [6]:
# Removing features
def feat_rm(df, feats):
    for feat in feats:
        df.drop(feat, axis = 1, inplace = True)
    
    return df

data = feat_rm(data, ['required_doughnuts_comsumption'])

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10728 entries, 0 to 10727
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   10728 non-null  int64 
 1   job_id               10728 non-null  int64 
 2   title                10728 non-null  object
 3   location             10523 non-null  object
 4   department           3719 non-null   object
 5   salary_range         1709 non-null   object
 6   company_profile      8766 non-null   object
 7   description          10727 non-null  object
 8   requirements         9099 non-null   object
 9   benefits             6360 non-null   object
 10  telecommuting        10728 non-null  int64 
 11  has_company_logo     10728 non-null  int64 
 12  has_questions        10728 non-null  int64 
 13  employment_type      8670 non-null   object
 14  required_experience  6489 non-null   object
 15  required_education   5827 non-null   object
 16  indu

In [8]:
data = data.replace(np.nan, '', regex=True)

In [9]:
data['text'] = data[['title', 'department','company_profile','description','requirements','benefits']].apply(lambda x: ' '.join(x), axis = 1)

In [10]:
data.drop(['job_id', 'location','title','salary_range' ,'department','salary_range','company_profile','description','requirements','benefits'], axis=1, inplace=True)

In [11]:
data.columns
data_columns = data.columns.tolist()

In [12]:
from sklearn.preprocessing import LabelEncoder
label_columns = ['telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function']
lb_make = LabelEncoder()
for i in label_columns:
  data[i] = lb_make.fit_transform(data[i])

In [13]:
data_columns = data_columns[-1:] + data_columns[:-1]
data = data[data_columns]
data.head()

,text,Id,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,Jr. Business Analyst & Quality Analyst (entry...,0,0,0,0,2,3,6,40,14,0
1,English Teacher Abroad We help teachers get ...,1,0,1,1,1,0,2,32,0,0
2,SQL Server Database Developer Job opportunity ...,2,0,0,0,2,6,2,58,19,0
3,Legal Analyst - 12 Month FTC Legal MarketInvoi...,3,0,1,0,2,1,7,40,20,0
4,Part-Time Finance Assistant Salary:£9 - £10 ...,4,0,0,0,4,0,0,1,0,0


In [14]:
# we are removing the words from the stop words list
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [15]:
## Defining the utility functions

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',str(text))


def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', str(text))


def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',str(text))


def remove_punctuation(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def final_preprocess(text):
    text = text.replace('\\r', ' ')
    text = text.replace('\\"', ' ')
    text = text.replace('\\n', ' ')
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    text = ' '.join(e for e in text.split() if e.lower() not in stopwords)
    text = text.lower()
    ps = PorterStemmer()
    text = ps.stem(text)
    return text
    

In [16]:
data['text']

0         Jr. Business Analyst & Quality Analyst (entry...
1        English Teacher Abroad   We help teachers get ...
2        SQL Server Database Developer Job opportunity ...
3        Legal Analyst - 12 Month FTC Legal MarketInvoi...
4        Part-Time Finance Assistant   Salary:£9 - £10 ...
                               ...                        
10723    Production Artist Design We're an emerging tec...
10724    iOS Developer  BlueLine Grid is transforming g...
10725    Graduates: English Teacher Abroad   We help te...
10726     Senior IT Security Lead (SIEM / IDS)  Roland ...
10727    Front-End Web Engineer  Joist is a platform fo...
Name: text, Length: 10728, dtype: object

In [17]:
#Cleaning

data['text'] = remove_URL(str(data['text']))
data['text'] = remove_emoji(str(data['text']))
data['text'] = remove_html(str(data['text']))
data['text'] = remove_punctuation(str(data['text']))
data['text'] = final_preprocess(str(data['text']))

In [18]:
data.head()

,text,Id,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,0 0 0 0 0 jr busine 1 0 0 0 0 jr busine 2 0 0 ...,0,0,0,0,2,3,6,40,14,0
1,0 0 0 0 0 jr busine 1 0 0 0 0 jr busine 2 0 0 ...,1,0,1,1,1,0,2,32,0,0
2,0 0 0 0 0 jr busine 1 0 0 0 0 jr busine 2 0 0 ...,2,0,0,0,2,6,2,58,19,0
3,0 0 0 0 0 jr busine 1 0 0 0 0 jr busine 2 0 0 ...,3,0,1,0,2,1,7,40,20,0
4,0 0 0 0 0 jr busine 1 0 0 0 0 jr busine 2 0 0 ...,4,0,0,0,4,0,0,1,0,0


In [ ]:
## Making a dictionary of the words and their vector representation

embeddings_index = {}
f = open('glove.840B.300d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    coefs = np.asarray(values[1:], dtype='float32') ## These are the vectors representing the embedding for the word
    embeddings_index[word] = coefs
f.close()

In [ ]:
#import com.johnsnowlabs.nlp.embeddings as comJ
#.WordEmbeddingsModel
#embeddings = comJ.WordEmbeddingsModel.pretrained("glove_840B_300", "xx")

In [27]:
#pip install johnsnowlabs